In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/nudt/.julia/packages/Plots/kyYZF/src/backends.jl:372


In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

1

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=-0.15
η2=0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=20/3*ν
γ2=40/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);


In [5]:
function sweep_detuning_300()
    ttot=300
    initial_x=[5.0 16]
    sweep_num=120
    start=30
    stop=150
    d=[]
    somega_g=[]
    somega_r=[]
    sn=[]
    for i in 1:sweep_num
        println(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],m))*instate
            nbose=real(tr(n*vec2dm(f)))
            print(" f ",nbose)
            return nbose
        end        

        a=optimize(feval,initial_x,LBFGS())
        initial_x=Optim.minimizer(a) 
        append!(somega_g,initial_x[1])  
        append!(somega_r,initial_x[2])
        append!(sn,Optim.minimum(a)[1])
        append!(d,m)
        df1=DataFrame(delta=d,omega_g=somega_g,omega_r=somega_r,n=sn);CSV.write("eit_sweep_detuning_300.csv", df1);
    end
    return somega_g,somega_r,sn,d
end
@time res=sweep_detuning_300()

1 
 f 0.3638535489436694 f 0.3638543982089491 f 0.3638650065501039 f 0.3638429408369372 f 0.36385397355876387 f 0.35088591974836686 f 0.35088671913908626 f 0.3508969510175702 f 0.35087568811027486 f 0.350886319427517 f 0.30305477239004786 f 0.303055351377225 f 0.3030641000370482 f 0.30304602395289393 f 0.30305506186795794 f 0.14503711022305119 f 0.14503600360540225 f 0.14504028348569345 f 0.14503283045498855 f 0.14503655691047054 f 0.8232768951492337 f 0.82329057170932 f 0.8232813756886751 f 0.8232860912237225 f 0.823283733459938 f 0.08832071333456282 f 0.0883197375813963 f 0.08832122200700473 f 0.08831922896915305 f 0.08832022545706748 f 0.07584448717807597 f 0.07584413758306283 f 0.07584396556429217 f 0.07584465926918596 f 0.07584431237986898 f 0.07584254693170811 f 0.07584218828579818 f 0.07584204168498714 f 0.07584269360456113 f 0.07584236760805649 f 0.072924993678902 f 0.07292461889899614 f 0.07292443865359001 f 0.07292517400272465 f 0.07292480628821282 f 0.06059879231941942 f 0.0

 f 0.05042497449971473 f 0.050425016139988156 f 0.050424638725495546 f 0.050425352058564625 f 0.05042499531562729 f 0.050398179281033254 f 0.05039821860390097 f 0.050397853841339595 f 0.05039854418781413 f 0.050398198938326054 f 0.05030016714037284 f 0.050300197468869386 f 0.05029988299313686 f 0.05030048176003557 f 0.050300182300510685 f 0.05002470790249594 f 0.050024699517048944 f 0.05002463010831478 f 0.05002477745544069 f 0.0500247037058379 f 0.05368723350583109 f 0.05368716602957808 f 0.05368830943023546 f 0.053686090293945214 f 0.053687199763628964 f 0.050009993883253494 f 0.05000997577284336 f 0.05000997852366098 f 0.05000999127721825 f 0.050009984824191216 f 0.05000487975649195 f 0.05000487401801647 f 0.05000487647873638 f 0.050004877442270376 f 0.05000487688323772 f 0.050041795755595544 f 0.05004184336996415 f 0.05004184524454921 f 0.05004179403539638 f 0.05004181955843896 f 0.05000436433855929 f 0.05000436365514027 f 0.05000436601505467 f 0.050004362126068465 f 0.050004363992

 f 0.042493548716261716 f 0.042493589027092146 f 0.04249322988270298 f 0.04249390801712665 f 0.04249356886784374 f 0.042472196016686094 f 0.042472234347634474 f 0.04247188652025612 f 0.04247254400042253 f 0.04247221517833382 f 0.04239366154401414 f 0.04239369217673312 f 0.042393389334909805 f 0.042393964541750484 f 0.042393676856628426 f 0.0421620402442266 f 0.04216203742418406 f 0.04216195385329371 f 0.0421621239708955 f 0.042162038830719784 f 0.04478420341426941 f 0.044784140756219566 f 0.044785136136542184 f 0.04478320823247289 f 0.04478417208171948 f 0.04214251701395664 f 0.042142502998844304 f 0.04214250532565706 f 0.042142514843556765 f 0.042142510002920235 f 0.04213928988929216 f 0.04213928460195366 f 0.04213928639599551 f 0.04213928825329822 f 0.04213928724215017 f 0.0421572360358478 f 0.042157267663339014 f 0.042157267644329685 f 0.04215723621966079 f 0.04215725184573191 f 0.04213878497876403 f 0.04213878455261161 f 0.0421387860678222 f 0.04213878362247529 f 0.0421387847621129

 f 0.03736688350875401 f 0.03736692251500273 f 0.037366575067158035 f 0.03736723112674111 f 0.03736690300833345 f 0.03734904970696949 f 0.037349086987845996 f 0.0373487500194808 f 0.03734938684536516 f 0.03734906834392873 f 0.037283212352677175 f 0.037283242914796025 f 0.03728294761638436 f 0.03728350782072562 f 0.03728322763036099 f 0.0370829657775967 f 0.03708296692694194 f 0.0370828749851041 f 0.037083057888631386 f 0.037082966349116044 f 0.03911694559350408 f 0.03911688902513791 f 0.039117796736580654 f 0.03911603809307122 f 0.039116917306088385 f 0.03706111995514643 f 0.03706110947684207 f 0.0370611115189261 f 0.037061118082965507 f 0.037061114712871046 f 0.03705926211566691 f 0.03705925749195527 f 0.03705925873234109 f 0.03705926104637026 f 0.03705925980058571 f 0.037066718497714396 f 0.03706673821212326 f 0.03706673636971946 f 0.037066720516794036 f 0.037066728351551996 f 0.037058823443952914 f 0.03705882323400987 f 0.037058823885929446 f 0.03705882296419499 f 0.0370588233357158

 f 0.03405244590249513 f 0.034052483676799236 f 0.03405214318204946 f 0.03405278658281024 f 0.03405246478646661 f 0.03403699329317918 f 0.03403702953533742 f 0.03403669900900506 f 0.034037324004958146 f 0.03403701141109404 f 0.03397980426648067 f 0.033979834535872194 f 0.03397954366552576 f 0.03398009532184052 f 0.03397981939814903 f 0.033802443837116024 f 0.03380244780248845 f 0.03380235083296621 f 0.0338025409909086 f 0.03380244581677052 f 0.03548537296056771 f 0.03548532205771148 f 0.03548618066146107 f 0.03548451458346563 f 0.03548534750630111 f 0.033779945829531144 f 0.03377993832317905 f 0.03377994006278183 f 0.033779944275131306 f 0.033779942073516186 f 0.03377898405004169 f 0.033778980270384014 f 0.03377898109964836 f 0.03377898340695676 f 0.03377898215726824 f 0.03378157244466925 f 0.03378158394584846 f 0.03378158116121416 f 0.03378157541955912 f 0.03378157819224066 f 0.03377865690751918 f 0.03377865684166988 f 0.03377865677525925 f 0.033778657161118185 f 0.03377865687169322 f

...

 f 0.06130325162661637 f 0.06130329070997989 f 0.0613028583145703 f 0.061303684676464035 f 0.06130327116641649 f 0.061294169757142546 f 0.06129420737361537 f 0.06129379039067793 f 0.06129458739482208 f 0.061294188563397814 f 0.06126114454200363 f 0.061261176370243905 f 0.061260821013227794 f 0.06126150055450727 f 0.061261160454054116 f 0.06117506981047738 f 0.06117507456046562 f 0.06117502712920957 f 0.06117511790390441 f 0.06117507218355944 f 0.06272458459038686 f 0.06272449757025593 f 0.06272602742449843 f 0.0627230554771931 f 0.06272454107882426 f 0.06117349780212043 f 0.06117349868167663 f 0.06117349834352592 f 0.06117349880399248 f 0.06117349824020155 f 0.06117349306513658 f 0.06117349384330249 f 0.06117349408864427 f 0.06117349348297657 f 0.06117349345231762 f 0.061173481826136876 f 0.061173482198620587 f 0.0611734847782041 f 0.06117347990962608 f 0.06117348201048774 f 0.06117361023016736 f 0.06117360857936009 f 0.06117362279906159 f 0.0611735966721467 f 0.0611736094031622 f 0.06

 f 0.0641695012215479 f 0.06416954078609051 f 0.06416910323603103 f 0.06416993945326716 f 0.06416952100188918 f 0.06416046653347413 f 0.06416050459647872 f 0.06416008282853211 f 0.06416088898311208 f 0.06416048556296429 f 0.06412765250692888 f 0.06412768464293998 f 0.06412732598277095 f 0.06412801185003489 f 0.06412766857310254 f 0.06404318504029496 f 0.06404318940293918 f 0.06404314612829382 f 0.06404322900478654 f 0.06404318721977091 f 0.0656147130461771 f 0.06561462223644449 f 0.0656161935039897 f 0.06561314254773086 f 0.06561466763976682 f 0.06404192669522063 f 0.06404192750754183 f 0.0640419272611171 f 0.06404192763279633 f 0.06404192709962192 f 0.06404192282478983 f 0.06404192354817378 f 0.06404192381367876 f 0.06404192325015795 f 0.06404192318469258 f 0.06404191313038506 f 0.0640419134992971 f 0.06404191581163236 f 0.06404191150879165 f 0.06404191331314898 f 0.06404200329544649 f 0.06404200189362345 f 0.06404201441768889 f 0.06404199146068532 f 0.06404200259285976 f 0.0640419117

 f 0.0671030625946129 f 0.06710310263047253 f 0.06710266007445571 f 0.06710350585950307 f 0.06710308261054555 f 0.06709407606802623 f 0.06709411456784933 f 0.06709368815755525 f 0.06709450318735377 f 0.06709409531606791 f 0.06706147528847532 f 0.0670615077237863 f 0.06706114587781771 f 0.06706183784483415 f 0.0670614915043992 f 0.06697857930931034 f 0.06697858328353679 f 0.06697854415054659 f 0.0669786191597112 f 0.06697858129455399 f 0.06857068207510413 f 0.06857058751344225 f 0.06857219940283098 f 0.06856907098296595 f 0.06857063479287835 f 0.06697758811905534 f 0.06697758886479417 f 0.06697758870690089 f 0.06697758899553324 f 0.0669775884901888 f 0.06697758498817419 f 0.06697758565685065 f 0.06697758594233204 f 0.0669775854212111 f 0.0669775853205551 f 0.06697757673766712 f 0.06697757709846203 f 0.06697757915676378 f 0.0669775753974886 f 0.06697757691629203 f 0.0669776378798218 f 0.06697763670377407 f 0.0669776476084733 f 0.06697762769183109 f 0.06697763728998321 f 0.066977575030627

 f 0.07009822600110588 f 0.0700982664948673 f 0.07009781908134394 f 0.07009867415064411 f 0.07009824624596894 f 0.07008928871673716 f 0.07008932764088523 f 0.07008889672991053 f 0.07008972036420134 f 0.07008930817697355 f 0.07005690360585701 f 0.07005693632969695 f 0.07005657141281543 f 0.07005726926043411 f 0.07005691996576634 f 0.06997554433158647 f 0.06997554791699034 f 0.06997551290142688 f 0.06997558009209923 f 0.06997554612235087 f 0.07158678655952504 f 0.07158668829669476 f 0.07158833998908318 f 0.07158513569279784 f 0.07158673742660449 f 0.06997477858225673 f 0.06997477926237185 f 0.06997477918925789 f 0.06997477940149995 f 0.06997477892048137 f 0.06997477607749313 f 0.06997477669135638 f 0.06997477699691194 f 0.06997477651798732 f 0.06997477638270959 f 0.06997476915662243 f 0.06997476950581703 f 0.06997477132497253 f 0.06997476808319296 f 0.06997476932939983 f 0.0699748087994636 f 0.06997480782660517 f 0.06997481720234089 f 0.06997480016819249 f 0.06997480831130619 f 0.0699747

(Any[2.921961896209877, 2.934477586063329, 2.947708573575306, 2.9616049390202837, 2.9761238748271444, 2.9912280704949707, 3.00688535780033, 3.0230670507140314, 3.039748116058111, 3.056905920626209  …  5.4925770313730835, 5.514268990467191, 5.535837932496586, 5.557279688630024, 5.578598757990095, 5.599789699765381, 5.620855627628392, 5.641793805795277, 5.662604532939297, 5.683288721603923], Any[11.0703281756456, 11.239465940814783, 11.406465512288847, 11.571387850652643, 11.73429148952703, 11.895233113913374, 12.054267686799045, 12.211447991569433, 12.366824755850104, 12.520447299324013  …  23.297687635235786, 23.379018503399568, 23.460069963083086, 23.540845791957864, 23.621348638721344, 23.701582601326727, 23.781550054753257, 23.861254638296717, 23.94069915171993, 24.019886799648944], Any[0.05951198079155564, 0.05677918828121257, 0.05430224519337525, 0.052052239074569184, 0.050004346270434166, 0.048137158371264345, 0.04643213439841284, 0.04487315267050434, 0.043446142193820744, 0.0421